# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
#Comment
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:\\Users\\Owner\\OneDrive\\Desktop\\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,15.95,73,25,4.97,AU,1708806151
1,1,avarua,-21.2078,-159.7750,26.03,100,75,0.51,CK,1708805937
2,2,tangalle,6.0240,80.7911,25.09,80,45,4.90,LK,1708806279
3,3,adamstown,-25.0660,-130.1015,26.45,77,53,5.92,PN,1708806158
4,4,vorkuta,67.5000,64.0000,-4.48,96,100,5.08,RU,1708806198


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = True, frame_width = 500, frame_height = 400, \
    size = city_data_df["Humidity"]/2)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
new_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] >= 16) & (city_data_df["Max Temp"] <= 20) & \
                                    (city_data_df["Humidity"] < 70) & (city_data_df["Wind Speed"] <= 3)]

new_city_data_df

# Drop any rows with null values
new_city_data_df = new_city_data_df.dropna(subset = ["Max Temp", "Wind Speed"])


# Display sample data
new_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,ribeira brava,32.6500,-17.0667,17.78,56,83,1.97,PT,1708806400
57,57,galeana,24.8333,-100.0667,19.57,48,97,1.70,MX,1708806417
99,99,tamanrasset,22.7850,5.5228,16.95,11,0,2.06,DZ,1708806439
183,183,summerlin south,36.1027,-115.3811,16.88,25,100,0.45,US,1708806501
216,216,chonchi,-42.6128,-73.8092,17.29,47,60,0.39,CL,1708806176
276,276,fortuna,40.5982,-124.1573,19.40,57,0,1.34,US,1708806198
299,299,debre werk',10.6667,38.1667,17.72,35,58,2.42,ET,1708806535
371,371,newman,37.3138,-121.0208,19.52,66,42,0.89,US,1708806298
389,389,bajitpur,24.2167,90.9500,18.13,64,49,2.88,BD,1708806572
422,422,ridgecrest,35.6225,-117.6709,18.69,29,0,1.34,US,1708806582


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_city_data_df.copy(deep = True)

hotel_df = hotel_df[["City_ID", "City", "Country", "Lat", "Lng", "Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
17,17,ribeira brava,PT,32.6500,-17.0667,56,
57,57,galeana,MX,24.8333,-100.0667,48,
99,99,tamanrasset,DZ,22.7850,5.5228,11,
183,183,summerlin south,US,36.1027,-115.3811,25,
216,216,chonchi,CL,-42.6128,-73.8092,47,
276,276,fortuna,US,40.5982,-124.1573,57,
299,299,debre werk',ET,10.6667,38.1667,35,
371,371,newman,US,37.3138,-121.0208,66,
389,389,bajitpur,BD,24.2167,90.9500,64,
422,422,ridgecrest,US,35.6225,-117.6709,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000


params = {
    "limit" : 1,
    "apiKey" : geoapify_key,
    "categories" : "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
galeana - nearest hotel: Hotel Jardin
tamanrasset - nearest hotel: فندق الأمان
summerlin south - nearest hotel: Element Las Vegas Summerline
chonchi - nearest hotel: Hotel Huildín
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
debre werk' - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
bajitpur - nearest hotel: No hotel found
ridgecrest - nearest hotel: Best Western China Lake Inn
beykonak - nearest hotel: Uyguluma Oteli
ivins - nearest hotel: Crescent Moon Inn


,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
17,17,ribeira brava,PT,32.6500,-17.0667,56,Cheerfulway Bravamar Hotel
57,57,galeana,MX,24.8333,-100.0667,48,Hotel Jardin
99,99,tamanrasset,DZ,22.7850,5.5228,11,فندق الأمان
183,183,summerlin south,US,36.1027,-115.3811,25,Element Las Vegas Summerline
216,216,chonchi,CL,-42.6128,-73.8092,47,Hotel Huildín
276,276,fortuna,US,40.5982,-124.1573,57,Comfort Inn & Suites Redwood Country
299,299,debre werk',ET,10.6667,38.1667,35,No hotel found
371,371,newman,US,37.3138,-121.0208,66,No hotel found
389,389,bajitpur,BD,24.2167,90.9500,64,No hotel found
422,422,ridgecrest,US,35.6225,-117.6709,29,Best Western China Lake Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = True, frame_width = 500, frame_height = 400, \
    size = (hotel_df["Humidity"]/2), hover_cols = ["Country","Hotel Name"])

# Display the map
hotel_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_size,Country,Hotel Name)